#**Pré-processamento**

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

import pandas as pd
import seaborn as sns
import missingno as msno
import warnings
from matplotlib import pyplot as plt
import datetime
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',100)

df = pd.read_csv("gdrive/My Drive/dissertação/Qualidade_do_ar_-_Dados_horarios.csv",sep=",")
df['Data']=df['Data'].astype('str')
df['Data']=df['Data'].apply(lambda x: x[:-3])

#conversão para datetime
def convert_to_datetime(x):
    return(datetime.datetime.strptime(x, "%Y/%m/%d %H:%M:%S"))
df['Data']=df['Data'].apply(convert_to_datetime) 
df['Hora-minuto']=df['Data'].apply(lambda x: str(x)[-8:-3])
# #extrair ano, mes, hora 
df['Ano']=df['Data'].apply(lambda x: x.year)
df['Mês']=df['Data'].apply(lambda x: x.month)
df['Dia']=df['Data'].apply(lambda x: x.day)
df['Hora']=df['Data'].apply(lambda x: x.hour)
df['Hora-minuto']=df['Data'].apply(lambda x: str(x)[-11:-6])

df_bg=df[df['Estação']=='BG'] #Bangu
df_gc=df[df['Estação']=='CG'] #Campo Grande
df_ca=df[df['Estação']=='CA'] #Centro
df_av=df[df['Estação']=='AV'] #Copacabana
df_ir=df[df['Estação']=='IR'] #Irajá
df_pg=df[df['Estação']=='PG'] #Pedra de Guaratiba
df_sc=df[df['Estação']=='SC'] #São Cristóvão
df_sp=df[df['Estação']=='SP'] #Tijuca

df_sc.set_index('Data',inplace=True)
df_sc.drop(columns=['OBJECTID','CodNum','Estação','Dir_Vento','NO2','HCNM','HCT','CH4','NO','NOx',\
                   'PM2_5', 'Lat', 'Lon', 'X_UTM_Sirgas2000','Y_UTM_Sirgas2000'], inplace=True)
df_sc_2=df_sc.copy() #para fazer distribuição sem eliminar ausentes

df_sc=df_sc['2011-01-01':'2018-01-01']
#explicar que usou interpolação linear com spline e justificar
df_sc['Chuva'].interpolate(method='slinear', inplace=True)
df_sc['Pres'].interpolate(method='slinear', inplace=True)
df_sc['RS'].interpolate(method='slinear', inplace=True)
df_sc['Temp'].interpolate(method='slinear', inplace=True)
df_sc['UR'].interpolate(method='slinear', inplace=True)
df_sc['Vel_Vento'].interpolate(method='slinear', inplace=True)
df_sc['SO2'].interpolate(method='slinear', inplace=True)
df_sc['CO'].interpolate(method='slinear', inplace=True)
df_sc['O3'].interpolate(method='slinear', inplace=True)
df_sc['PM10'].interpolate(method='slinear', inplace=True)

Mounted at /content/gdrive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#**VAR LAG=1**

In [90]:
df_sc_2=df_sc.copy()
df_sc_2.drop(columns=['Hora-minuto', 'Ano', 'Mês', 'Dia', 'Hora'], inplace=True)

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [92]:
nobs = 15841
df_train, df_test = df_sc_2[0:-nobs], df_sc_2[-nobs:]

# Check size
print(df_train.shape)  # (119, 8)
print(df_test.shape)  # (1, 8)

(44520, 10)
(15841, 10)


In [93]:
model = VAR(df_train)

In [94]:
model_fitted = model.fit(1) #aqui vem o LAG

In [95]:
# Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

1


In [96]:
forecast_input = df_sc_2.values[-lag_order:]

In [97]:
# Forecast
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df_sc_2.index[-nobs:], columns=df_sc_2.columns)
df_forecast

,Chuva,Pres,RS,Temp,UR,Vel_Vento,SO2,CO,O3,PM10
Data,,,,,,,,,,
2016-02-23 17:30:00,0.091257,1009.356024,5.510346,27.598200,69.742575,0.438796,7.824190,0.143817,18.718480,25.541256
2016-02-23 18:30:00,0.125562,1009.488541,12.044198,27.354571,70.940008,0.528026,8.001442,0.202517,15.693471,27.259039
2016-02-23 19:30:00,0.137549,1009.631350,21.026633,27.156708,71.937362,0.578171,8.242882,0.249387,13.430512,29.055630
2016-02-23 20:30:00,0.140255,1009.773442,32.344587,27.012953,72.709891,0.607624,8.530368,0.286751,11.873609,30.845535
2016-02-23 21:30:00,0.138905,1009.907498,45.563174,26.925421,73.253584,0.627319,8.839691,0.316332,10.954427,32.562892
...,...,...,...,...,...,...,...,...,...,...
2018-01-01 19:30:00,0.099658,1014.822393,152.409742,27.544486,74.195378,0.894689,8.367884,0.308575,21.136909,37.613959
2018-01-01 20:30:00,0.099658,1014.822393,152.409742,27.544486,74.195378,0.894689,8.367884,0.308575,21.136909,37.613959
2018-01-01 21:30:00,0.099658,1014.822393,152.409742,27.544486,74.195378,0.894689,8.367884,0.308575,21.136909,37.613959


In [98]:
from statsmodels.tsa.stattools import acf
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'corr':corr, 'minmax':minmax})

print('Forecast Accuracy of: O3')
accuracy_prod = forecast_accuracy(df_forecast['O3'].values, df_test['O3'])
for k, v in accuracy_prod.items():
    print(k, ': ', round(v,4))

Forecast Accuracy of: O3
mape :  inf
me :  -0.817
mae :  18.5502
mpe :  inf
rmse :  23.9265
corr :  0.0073
minmax :  0.598


In [99]:
from sklearn.metrics import r2_score #colocar citação
# calculate R2
r2=r2_score(df_test['O3'], df_forecast['O3'],multioutput='uniform_average')
print('Test R2: %.3f' % r2)

Test R2: -0.001


#**VAR LAG=2**

In [101]:
df_sc_2=df_sc.copy()
df_sc_2.drop(columns=['Hora-minuto', 'Ano', 'Mês', 'Dia', 'Hora'], inplace=True)

In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [103]:
nobs = 15841
df_train, df_test = df_sc_2[0:-nobs], df_sc_2[-nobs:]

# Check size
print(df_train.shape)  # (119, 8)
print(df_test.shape)  # (1, 8)

(44520, 10)
(15841, 10)


In [104]:
model = VAR(df_train)

In [105]:
model_fitted = model.fit(2) #aqui vem o LAG

In [106]:
# Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

2


In [107]:
forecast_input = df_sc_2.values[-lag_order:]

In [108]:
# Forecast
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df_sc_2.index[-nobs:], columns=df_sc_2.columns)
df_forecast

,Chuva,Pres,RS,Temp,UR,Vel_Vento,SO2,CO,O3,PM10
Data,,,,,,,,,,
2016-02-23 17:30:00,0.052235,1009.056651,-12.409145,27.391939,70.908931,0.372035,7.607981,0.117289,18.527823,25.515621
2016-02-23 18:30:00,0.092235,1009.055621,-15.531767,26.999738,72.726266,0.429732,7.686156,0.181513,15.047775,27.731272
2016-02-23 19:30:00,0.120213,1009.185105,-2.502476,26.774714,73.824406,0.471485,8.090277,0.238119,12.546145,30.022071
2016-02-23 20:30:00,0.136751,1009.386738,24.611760,26.732020,74.223523,0.507056,8.703029,0.282411,11.275313,32.341056
2016-02-23 21:30:00,0.143993,1009.613487,60.917223,26.855015,74.002277,0.546429,9.384563,0.314733,11.255960,34.538612
...,...,...,...,...,...,...,...,...,...,...
2018-01-01 19:30:00,0.099705,1014.816476,152.341870,27.544437,74.201675,0.894486,8.370076,0.308705,21.128054,37.620232
2018-01-01 20:30:00,0.099705,1014.816476,152.341870,27.544437,74.201675,0.894486,8.370076,0.308705,21.128054,37.620232
2018-01-01 21:30:00,0.099705,1014.816476,152.341870,27.544437,74.201675,0.894486,8.370076,0.308705,21.128054,37.620232


In [109]:
from statsmodels.tsa.stattools import acf
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'corr':corr, 'minmax':minmax})

print('Forecast Accuracy of: O3')
accuracy_prod = forecast_accuracy(df_forecast['O3'].values, df_test['O3'])
for k, v in accuracy_prod.items():
    print(k, ': ', round(v,4))

Forecast Accuracy of: O3
mape :  inf
me :  -0.8062
mae :  18.557
mpe :  inf
rmse :  23.926
corr :  0.0077
minmax :  0.5981


In [110]:
from sklearn.metrics import r2_score #colocar citação
# calculate R2
r2=r2_score(df_test['O3'], df_forecast['O3'],multioutput='uniform_average')
print('Test R2: %.3f' % r2)

Test R2: -0.001


#**VAR LAG=3**

In [112]:
df_sc_2=df_sc.copy()
df_sc_2.drop(columns=['Hora-minuto', 'Ano', 'Mês', 'Dia', 'Hora'], inplace=True)

In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [114]:
nobs = 15841
df_train, df_test = df_sc_2[0:-nobs], df_sc_2[-nobs:]

# Check size
print(df_train.shape)  # (119, 8)
print(df_test.shape)  # (1, 8)

(44520, 10)
(15841, 10)


In [115]:
model = VAR(df_train)

In [116]:
model_fitted = model.fit(3) #aqui vem o LAG

In [117]:
# Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

3


In [118]:
forecast_input = df_sc_2.values[-lag_order:]

In [119]:
# Forecast
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df_sc_2.index[-nobs:], columns=df_sc_2.columns)
df_forecast

,Chuva,Pres,RS,Temp,UR,Vel_Vento,SO2,CO,O3,PM10
Data,,,,,,,,,,
2016-02-23 17:30:00,0.050422,1008.909338,19.858811,27.572834,70.321993,0.383308,7.990094,0.110356,20.396358,24.762573
2016-02-23 18:30:00,0.080540,1008.793354,17.566478,27.281730,71.664922,0.428522,7.802231,0.163591,17.816699,26.679888
2016-02-23 19:30:00,0.106075,1008.865262,22.688313,27.076999,72.601994,0.478574,8.044649,0.217862,15.629933,28.771051
2016-02-23 20:30:00,0.128849,1009.049236,42.815021,27.029367,72.930547,0.535224,8.584790,0.262615,14.243004,30.966849
2016-02-23 21:30:00,0.140951,1009.270244,77.565739,27.163624,72.573015,0.585815,9.253471,0.294249,14.051360,33.120833
...,...,...,...,...,...,...,...,...,...,...
2018-01-01 19:30:00,0.099655,1014.820908,152.303720,27.540928,74.201815,0.894390,8.369001,0.308708,21.123427,37.618749
2018-01-01 20:30:00,0.099655,1014.820908,152.303720,27.540928,74.201815,0.894390,8.369001,0.308708,21.123427,37.618749
2018-01-01 21:30:00,0.099655,1014.820908,152.303720,27.540928,74.201815,0.894390,8.369001,0.308708,21.123427,37.618749


In [120]:
from statsmodels.tsa.stattools import acf
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'corr':corr, 'minmax':minmax})

print('Forecast Accuracy of: O3')
accuracy_prod = forecast_accuracy(df_forecast['O3'].values, df_test['O3'])
for k, v in accuracy_prod.items():
    print(k, ': ', round(v,4))

Forecast Accuracy of: O3
mape :  inf
me :  -0.7984
mae :  18.5601
mpe :  inf
rmse :  23.9248
corr :  0.0116
minmax :  0.5981


In [121]:
from sklearn.metrics import r2_score #colocar citação
# calculate R2
r2=r2_score(df_test['O3'], df_forecast['O3'],multioutput='uniform_average')
print('Test R2: %.3f' % r2)

Test R2: -0.001


#**VAR LAG=4**

In [75]:
df_sc_2=df_sc.copy()
df_sc_2.drop(columns=['Hora-minuto', 'Ano', 'Mês', 'Dia', 'Hora'], inplace=True)

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [81]:
nobs = 15841 #44520
df_train, df_test = df_sc_2[0:-nobs], df_sc_2[-nobs:]

# Check size
print(df_train.shape)  # (119, 8)
# print(df_test.shape)  # (1, 8)

(44520, 10)


In [82]:
model = VAR(df_train)

In [83]:
model_fitted = model.fit(4) #aqui vem o LAG

In [84]:
# Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

4


In [85]:
forecast_input = df_sc_2.values[-lag_order:]

In [86]:
# Forecast
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df_sc_2.index[-nobs:], columns=df_sc_2.columns)
df_forecast

,Chuva,Pres,RS,Temp,UR,Vel_Vento,SO2,CO,O3,PM10
Data,,,,,,,,,,
2016-02-23 17:30:00,0.062874,1008.825016,21.549797,27.538590,70.490436,0.378540,7.835496,0.100011,21.336920,24.406337
2016-02-23 18:30:00,0.110574,1008.517626,31.358494,27.291219,71.681886,0.428506,7.364669,0.135764,20.581445,25.388811
2016-02-23 19:30:00,0.096417,1008.440014,34.455214,27.120810,72.512054,0.462223,7.543230,0.184433,18.382892,26.718632
2016-02-23 20:30:00,0.114793,1008.554461,48.648411,27.077552,72.844740,0.515822,8.043935,0.235665,16.627892,28.963396
2016-02-23 21:30:00,0.134022,1008.785995,77.556791,27.205217,72.489879,0.580583,8.758411,0.278107,15.754851,31.376468
...,...,...,...,...,...,...,...,...,...,...
2018-01-01 19:30:00,0.099584,1014.826326,152.281135,27.538288,74.202670,0.894313,8.369378,0.308734,21.119287,37.624119
2018-01-01 20:30:00,0.099584,1014.826326,152.281135,27.538288,74.202670,0.894313,8.369378,0.308734,21.119287,37.624119
2018-01-01 21:30:00,0.099584,1014.826326,152.281135,27.538288,74.202670,0.894313,8.369378,0.308734,21.119287,37.624119


In [87]:
from statsmodels.tsa.stattools import acf
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'corr':corr, 'minmax':minmax})

print('Forecast Accuracy of: O3')
accuracy_prod = forecast_accuracy(df_forecast['O3'].values, df_test['O3'])
for k, v in accuracy_prod.items():
    print(k, ': ', round(v,4))

Forecast Accuracy of: O3
mape :  inf
me :  -0.7857
mae :  18.565
mpe :  inf
rmse :  23.9239
corr :  0.0133
minmax :  0.5982


In [88]:
from sklearn.metrics import r2_score #colocar citação
# calculate R2
r2=r2_score(df_test['O3'], df_forecast['O3'],multioutput='uniform_average')
print('Test R2: %.3f' % r2)

Test R2: -0.001


#**Teste**

In [ ]:
df_sc_2=df_sc.copy()
df_sc_2.drop(columns=['Hora-minuto', 'Ano', 'Mês', 'Dia', 'Hora'], inplace=True)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic